# NOTE: this HOWTO has been replaced by the `tf.Estimator` API

---



# HowTo: running `train` & `validation` loops in the same session with `tf-slim`

`Validation` loops are used to monitor `training` to identify when models reach a high variance state, e.g. `overfitting`. 

The purpose of this notebook is to provide an example of how to run a `validation` loop in the same session as a `train` loop using `tf.slim`. We'll also provide an example on how to show `validation` summaries on `tensorboard`.



## references:

* This example borrows from the [slim walkthough](https://github.com/tensorflow/models/blob/master/research/slim/slim_walkthrough.ipynb) notebook  
* recipe for [running train/validation/test loops](https://github.com/tensorflow/tensorflow/issues/5987) in the same `tf.slim` session
* plot [training and validation losses](https://stackoverflow.com/questions/37146614/tensorboard-plot-training-and-validation-losses-on-the-same-graph)  on the same tensorboard graph




# Simple Example

## Setup

In [0]:
HOME = "/content"
SLIM = "/content/models/research/slim"
DATA_DIR = "/tmp/tfrecords"

import os
# load repo for TF-Slim image models
found = os.path.isdir('/content/models')
if not found:
  !git clone https://github.com/tensorflow/models.git
  !git clone https://github.com/mixuala/colab_utils.git

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, shutil

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time

from models.research.slim.datasets import dataset_utils
from colab_utils import tboard

# Main slim library
from tensorflow.contrib import slim

## Build `tf.slim` training loop

In [0]:
### 
### dataset pre-processing and tensorboard methods
###
def produce_batch(batch_size, noise=0.3):
    xs = np.random.random(size=[batch_size, 1]) * 10
    ys = np.sin(xs) + 5 + np.random.normal(size=[batch_size, 1], scale=noise)
    return [xs.astype(np.float32), ys.astype(np.float32)]

def convert_data_to_tensors(x, y):
    inputs = tf.constant(x)
    inputs.set_shape([None, 1])
    
    outputs = tf.constant(y)
    outputs.set_shape([None, 1])
    return inputs, outputs
  
  

def reset_tensorboard(log_dir):
  try:
    shutil.rmtree(log_dir)
  except:
    pass
  if not tf.gfile.Exists(log_dir):  tf.gfile.MakeDirs(log_dir)  
  not_empty = %ls $log_dir
  if not not_empty: print("TRAIN_DIR={}, ls:{}".format(log_dir, not_empty))

In [0]:
### 
### net
###
def regression_model(inputs, is_training=True, scope="regression_model"):
    """Creates the regression model.

    Args:
        inputs: A node that yields a `Tensor` of size [batch_size, dimensions].
        is_training: Whether or not we're currently training the model.
        scope: An optional variable_op scope for the model.

    Returns:
        predictions: 1-D `Tensor` of shape [batch_size] of responses.
        end_points: A dict of end points representing the hidden layers.
    """
    # Make the model, reuse weights for validation batches using reuse=tf.AUTO_REUSE
    with slim.arg_scope([slim.fully_connected], reuse=tf.AUTO_REUSE):
        end_points = {}
        # Set the default weight _regularizer and acvitation for each fully_connected layer.
        with slim.arg_scope([slim.fully_connected],
                            activation_fn=tf.nn.relu,
                            weights_regularizer=slim.l2_regularizer(0.01)):

            # Creates a fully connected layer from the inputs with 32 hidden units.
            net = slim.fully_connected(inputs, 32, scope='fc1')
            end_points['fc1'] = net

            # Adds a dropout layer to prevent over-fitting.
            net = slim.dropout(net, 0.8, is_training=is_training)

            # Adds another fully connected layer with 16 hidden units.
            net = slim.fully_connected(net, 16, scope='fc2')
            end_points['fc2'] = net

            # Creates a fully-connected layer with a single hidden unit. Note that the
            # layer is made linear by setting activation_fn=None.
            predictions = slim.fully_connected(net, 1, activation_fn=None, scope='prediction')
            end_points['out'] = predictions

    return predictions, end_points


### add `validation` loop to `train_step_fn()`
validate on given step interval

In [0]:

# 
def train_step_fn(sess, train_op, global_step, train_step_kwargs):
  """
  slim.learning.train_step():
    train_step_kwargs = {summary_writer:, should_log:, should_stop:}

  usage: slim.learning.train( train_op, logdir, 
                              train_step_fn=train_step_fn,)
  """
  if hasattr(train_step_fn, 'step'):
    train_step_fn.step += 1  # or use global_step.eval(session=sess)
  else:
    train_step_fn.step = global_step.eval(sess)
  
  # calc training losses
  total_loss, should_stop = slim.learning.train_step(sess, train_op, global_step, train_step_kwargs)
  
  
  # validate on interval
  if train_step_fn.step % VALIDATION_INTERVAL == 0:
    validiate_loss, validation_delta = sess.run([val_loss, summary_validation_delta])
    print(">> global step {}:    train={}   validation={}  delta={}".format(train_step_fn.step, 
                        total_loss, validiate_loss, validiate_loss-total_loss))
    

  return [total_loss, should_stop]

## Using `TFRecords` with `slim.dataset.Dataset`

see: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim/python/slim/data/

reading data: 
- https://www.tensorflow.org/api_guides/python/reading_data
- https://github.com/kwotsin/transfer_learning_tutorial/blob/master/train_flowers.py

#### get inputs, targets manually from tf.data.TFRecordDataset()

In [112]:
### using TFRecordDataset directly
import os
# example proto decode
def _parse_function(example_proto):
  keys_to_features = {'input': tf.FixedLenFeature((), tf.float32),
                      'label': tf.FixedLenFeature((), tf.float32)
                     }
  parsed_features = tf.parse_single_example(example_proto, keys_to_features)
  return parsed_features['input'], parsed_features['label']

train_sources = ["{}/{}".format(DATA_DIR, f) for f in os.listdir(DATA_DIR) if "train" in f]
train_dataset = tf.data.TFRecordDataset(train_sources)
train_dataset = train_dataset.map(_parse_function).repeat().batch(32)

print( type(train_dataset))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


### create `TFRecord` files

In [113]:
### create TFRecord files from produce_batch()

if not tf.gfile.Exists(DATA_DIR):  tf.gfile.MakeDirs(DATA_DIR)  

def np_to_tfrecord(inputs, labels, filename, data_dir):  
  
  writer = tf.python_io.TFRecordWriter( "{}/{}.tfrecord".format(data_dir, filename))
  for m in range(len(inputs)):
    # Feature contains a map of string for each feature proto objects
    # NOTE: for tf.train.FloatList|Int64List value must be a tuple or list!!
    feature = {
        "input": tf.train.Feature(float_list=tf.train.FloatList(value= inputs[m].tolist() )),
        "label": tf.train.Feature(float_list=tf.train.FloatList(value= labels[m].tolist() )),
    }

    # Construct the Example proto object
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    # Serialize the example to a string, write records to a tfrecords file

    writer.write(example.SerializeToString())
    
  writer.close()

  
# create ".tfrecord" files  
for i in range(4):  
  x, y = produce_batch(320)    # x.shape=[320,1], y.shape=[320,1]
  np_to_tfrecord(x, y, "train_{}".format(i), DATA_DIR)
  x, y = produce_batch(64)    # x.shape=[320,1], y.shape=[320,1]
  np_to_tfrecord(x, y, "validation_{}".format(i), DATA_DIR)

%ls $DATA_DIR


train_0.tfrecord  train_3.tfrecord       validation_2.tfrecord
train_1.tfrecord  validation_0.tfrecord  validation_3.tfrecord
train_2.tfrecord  validation_1.tfrecord


### read the `TFRecords` using the Dataset API

In [114]:
### using slim.dataset.Dataset
import os

### helpers
def _count_tfrecord_samples(tfrecord_list):
  """Count the total number of examples in list of tfrecord files"""
  num_samples = 0
  for tfrecord_file in tfrecord_list:
      print(tfrecord_file)
      for record in tf.python_io.tf_record_iterator(tfrecord_file):
          num_samples += 1
  return num_samples

def load_batch_from_slim_dataset(dataset, batch_size=32):
  """usage:
        x_train, y_train = load_batch_from_slim_dataset(train_dataset)
        x_validation, y_validation = load_batch_from_slim_dataset(validation_dataset)  
  """
  data_provider = slim.dataset_data_provider.DatasetDataProvider(
                  dataset, common_queue_capacity=32,
                  common_queue_min=8)
  input, label = data_provider.get(['input', 'label'])
  
  # Batch it up.
  inputs, labels = tf.train.batch(
        [input, label],
        batch_size=batch_size,
        num_threads=1,
        capacity=2 * batch_size)

  return [inputs, labels]

  




### create slim datasets for train/validation
keys_to_features = {
    'input': tf.FixedLenFeature( [], tf.float32),
    'label': tf.FixedLenFeature( [], tf.float32),
}

items_to_handlers = {
    'input': slim.tfexample_decoder.Tensor('input'),
    'label': slim.tfexample_decoder.Tensor('label'),
}

decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

train_sources = ["{}/{}".format(DATA_DIR, f) for f in os.listdir(DATA_DIR) if "train" in f]
# train_sources = "{}_*.tfrecord".format("train")
train_dataset = slim.dataset.Dataset(
                              data_sources=train_sources,    
                              reader=tf.TFRecordReader,
                              decoder=decoder,
                              num_samples=_count_tfrecord_samples(train_sources),
                              items_to_descriptions={})


validation_sources = ["{}/{}".format(DATA_DIR, f) for f in os.listdir(DATA_DIR) if "validation" in f]
# validation_sources = "{}_*.tfrecord".format("validation")
validation_dataset = slim.dataset.Dataset(
                              data_sources=validation_sources,    
                              reader=tf.TFRecordReader,
                              decoder=decoder,
                              num_samples=_count_tfrecord_samples(validation_sources),
                              items_to_descriptions={})

print( type(train_dataset))

/tmp/tfrecords/train_3.tfrecord
/tmp/tfrecords/train_2.tfrecord
/tmp/tfrecords/train_0.tfrecord
/tmp/tfrecords/train_1.tfrecord
/tmp/tfrecords/validation_1.tfrecord
/tmp/tfrecords/validation_0.tfrecord
/tmp/tfrecords/validation_2.tfrecord
/tmp/tfrecords/validation_3.tfrecord
<class 'tensorflow.contrib.slim.python.slim.data.dataset.Dataset'>


## Build graph for training loop

In [110]:
### 
### runtime params
###
LOG_DIR = '/tmp/tensorboard'
TRAIN_DIR = LOG_DIR + "/train"
LOG_INTERVAL = 100
VALIDATION_INTERVAL = 500
STEPS = 1000
tboard.launch_tensorboard(bin_dir="/tmp", log_dir=LOG_DIR)
tf.logging.set_verbosity(tf.logging.INFO)

!ls $TRAIN_DIR
print(TRAIN_DIR)



ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://a62b9279.ngrok.io
ls: cannot access '/tmp/tensorboard/train': No such file or directory
/tmp/tensorboard/train


In [116]:

## graph
with tf.Graph().as_default():
  
  if "TFRecordDataset" and False:
    ### get inputs, targets manually from tf.data.TFRecordDataset()
    batch_size = 32
    x_train, y_train = produce_batch(batch_size)
    x_validation, y_validation = produce_batch(batch_size)
    inputs, targets = convert_data_to_tensors(x_train, y_train)
    val_inputs, val_targets = convert_data_to_tensors(x_validation, y_validation)
    print("inputs/targets:", inputs,targets)
   
  else:  
    ### get inputs, targets from slim.data.dataset.Dataset()
    inputs, targets = load_batch_from_slim_dataset(train_dataset)
    print("inputs:", inputs, "targets:", targets)
    inputs = tf.reshape(inputs, [-1,1])
    targets = tf.reshape(targets, [-1,1])
#     print("inputs:", inputs, "targets:", targets)
    
    val_inputs, val_targets = load_batch_from_slim_dataset(validation_dataset)
    val_inputs = tf.reshape(val_inputs, [-1,1])
    val_targets = tf.reshape(val_targets, [-1,1])
    
    

  predictions, nodes = regression_model(inputs, is_training=True)
  val_predictions, _ = regression_model(val_inputs, is_training=False)
    

  ###
  ### train graph
  ###
  # with tf.variable_scope("train"):
  loss = tf.losses.mean_squared_error(labels=targets, predictions=predictions)
  
  ###
  ### validation graph
  ###     evaluate from `train_step_fn()`, usually after each epoch
  ###
  # careful, use different loss_collection so you don't add validation losses to training losses
  val_loss = tf.losses.mean_squared_error(labels=val_targets, predictions=val_predictions,
                                          loss_collection="validation" 
                                         )
                                         
  ### train_op
  total_loss = tf.losses.get_total_loss()       # excludes loss_collection="validation"
  optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
  train_op = slim.learning.create_train_op(total_loss, optimizer)
  print("\n >> total_loss=", total_loss)  
  
  
  ### add summaries
  # train summaries
  summary_loss = tf.summary.scalar("train/loss", total_loss)
  train_writer = tf.summary.FileWriter(TRAIN_DIR)

  # validation summaries
  summary_validation_loss = tf.summary.scalar("validation/loss", val_loss )
  summary_validation_delta = tf.summary.scalar("validation/loss_delta", (val_loss-loss) )                          
  print("\n >> validation losses=", tf.losses.get_losses(loss_collection="validation"))


  
  if True: reset_tensorboard(TRAIN_DIR)
    
    
  tboard.launch_tensorboard(log_dir=LOG_DIR)

  # Run the training inside a session.
  final_loss = slim.learning.train(
      train_op,
      train_step_fn=train_step_fn,
      logdir=LOG_DIR,
      number_of_steps=STEPS,
#       summary_op=train_summary_op,
      summary_writer=train_writer,
      save_summaries_secs=10,
      save_interval_secs=1000,
      log_every_n_steps=LOG_INTERVAL,
      )
  
print("Finished training. Last batch loss:", final_loss)
print("Checkpoint saved in %s" % LOG_DIR)

inputs: Tensor("batch:0", shape=(32,), dtype=float32) targets: Tensor("batch:1", shape=(32,), dtype=float32)

 >> total_loss= Tensor("total_loss:0", shape=(), dtype=float32)

 >> validation losses= [<tf.Tensor 'mean_squared_error_1/value:0' shape=() dtype=float32>]
TRAIN_DIR=/tmp/tensorboard/train, ls:None
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://a62b9279.ngrok.io
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /tmp/tensorboard/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:Recording summary at step 0.
>> global step 0:    train=27.787139892578125   validation=23.375640869140625  delta=-4.4114990234375
INFO:tensorflow:global step 100: loss = 1.3815 (0.016 sec/step)
INFO:tensorflow:global step 200: loss = 1.0201 (0.019 sec/step)
INFO:tensorflow:global step 300: loss = 0.9106 (0.018 sec/step)
INFO:tensorflow:global step 400: loss 

INFO:tensorflow:global step 3200: loss = 0.1972 (0.017 sec/step)
INFO:tensorflow:global step 3300: loss = 0.2573 (0.016 sec/step)
INFO:tensorflow:global step 3400: loss = 0.2848 (0.021 sec/step)
INFO:tensorflow:Recording summary at step 3466.
INFO:tensorflow:global step 3500: loss = 0.2187 (0.018 sec/step)
>> global step 3500:    train=0.25522756576538086   validation=0.17607265710830688  delta=-0.07915490865707397
INFO:tensorflow:global step 3600: loss = 0.2358 (0.019 sec/step)
INFO:tensorflow:global step 3700: loss = 0.2916 (0.032 sec/step)
INFO:tensorflow:global step 3800: loss = 0.2555 (0.018 sec/step)
INFO:tensorflow:global step 3900: loss = 0.2501 (0.016 sec/step)
INFO:tensorflow:Recording summary at step 3962.
INFO:tensorflow:global step 4000: loss = 0.2296 (0.017 sec/step)
>> global step 4000:    train=0.2301236093044281   validation=0.15709303319454193  delta=-0.07303057610988617
INFO:tensorflow:global step 4100: loss = 0.2411 (0.018 sec/step)
INFO:tensorflow:global step 4200:

INFO:tensorflow:global step 7000: loss = 0.2393 (0.016 sec/step)
>> global step 7000:    train=0.15158812701702118   validation=0.12626442313194275  delta=-0.02532370388507843
INFO:tensorflow:global step 7100: loss = 0.1524 (0.017 sec/step)
INFO:tensorflow:global step 7200: loss = 0.1922 (0.019 sec/step)
INFO:tensorflow:global step 7300: loss = 0.1835 (0.018 sec/step)
INFO:tensorflow:global step 7400: loss = 0.2123 (0.018 sec/step)
INFO:tensorflow:Recording summary at step 7459.
INFO:tensorflow:global step 7500: loss = 0.2090 (0.015 sec/step)
>> global step 7500:    train=0.23725485801696777   validation=0.10145480185747147  delta=-0.1358000636100769
INFO:tensorflow:global step 7600: loss = 0.1827 (0.018 sec/step)
INFO:tensorflow:global step 7700: loss = 0.2275 (0.017 sec/step)
INFO:tensorflow:global step 7800: loss = 0.2638 (0.018 sec/step)
INFO:tensorflow:global step 7900: loss = 0.2335 (0.017 sec/step)
INFO:tensorflow:Recording summary at step 7970.
INFO:tensorflow:global step 8000:

# Flowers example using `TFRecord` datasets
uses the flowers dataset from slim models

/content/models/research/slim


In [118]:
%cd $SLIM
from preprocessing import inception_preprocessing
import tensorflow as tf

from tensorflow.contrib import slim


def load_batch(dataset, batch_size=32, height=299, width=299, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])
    
    # Preprocess image for usage by Inception.
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels = tf.train.batch(
          [image, image_raw, label],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size)
    
    return images, images_raw, labels

/content/models/research/slim


In [0]:
## runtime params
VALIDATION_INTERVAL = 1

# colab GPU adjustments
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True

In [0]:
train_step = slim.learning.train_step

# slim.learning.train(train_step_fn=)
def train_step_fn(sess, train_op, global_step, train_step_kwargs):
  """
  slim.learning.train_step():
    train_step_kwargs = {summary_writer:, should_log:, should_stop:}
  """
  if hasattr(train_step_fn, 'step'):
    train_step_fn.step += 1  # or use global_step.eval(session=sess)
  else:
    train_step_fn.step = global_step.eval(sess)
  
  # calc training losses
  total_loss, should_stop = train_step(sess, train_op, global_step, train_step_kwargs)
  
  
  # validate on interval
  if train_step_fn.step and train_step_fn.step % VALIDATION_INTERVAL == 0:
    np_train_loss, np_val_loss, _ = sess.run([train_loss, val_loss, summary_validation_delta])
    print(">> global step {}:    train={}   validation={}  delta={}".format(train_step_fn.step, 
                        np_train_loss, np_val_loss, np_val_loss-np_train_loss))
    

  return [total_loss, should_stop]

In [0]:
def my_cnn(images, num_classes, is_training):  # is_training is not used...
    with slim.arg_scope([slim.conv2d, slim.fully_connected], reuse=tf.AUTO_REUSE):
      with slim.arg_scope([slim.max_pool2d], kernel_size=[3, 3], stride=2):
          net = slim.conv2d(images, 64, [5, 5], scope="conv1")
          net = slim.max_pool2d(net)
          net = slim.conv2d(net, 64, [5, 5], scope="conv2")
          net = slim.max_pool2d(net)
          net = slim.flatten(net)
          net = slim.fully_connected(net, 192, scope="fc1")
          net = slim.fully_connected(net, num_classes, activation_fn=None, scope="fc2")       
          return net

## training loop

In [122]:
from datasets import flowers

# This might take a few minutes.
LOG_DIR = '/tmp/tensorboard'
TRAIN_DIR = LOG_DIR + "/flowers"
print('Will save model to %s' % TRAIN_DIR)

tboard.launch_tensorboard(bin_dir="/tmp", log_dir=LOG_DIR)

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)

    dataset = flowers.get_split('train', flowers_data_dir)
    images, _, labels = load_batch(dataset)
    print("inputs:", images, "targets:", labels)

  
    val_dataset = flowers.get_split('validation', flowers_data_dir)
    val_images, _, val_labels = load_batch(val_dataset)
    
    
    # Create the model:
    logits = my_cnn(images, num_classes=dataset.num_classes, is_training=True)
    val_logits = my_cnn(val_images, num_classes=dataset.num_classes, is_training=False)
 
    # Specify the `train` loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    train_loss = tf.losses.softmax_cross_entropy (one_hot_labels, logits)
    total_loss = slim.losses.get_total_loss()
    
    # Specify the `validation` loss function:
    val_one_hot_labels = slim.one_hot_encoding(val_labels, dataset.num_classes)
    val_loss = tf.losses.softmax_cross_entropy (val_one_hot_labels, val_logits, 
                                     loss_collection="validation")
    

    # Create some summaries to visualize the training process:
    tf.summary.scalar('train/Total_Loss', total_loss)
    tf.summary.scalar('validation/Validation_Loss', val_loss)
    summary_validation_delta = tf.summary.scalar('validation/Validation_Delta', (val_loss - train_loss))
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)

    if True: reset_tensorboard(LOG_DIR)
        
    # Run the training:
    final_loss = slim.learning.train(
      train_op,
      train_step_fn=train_step_fn,
      logdir=TRAIN_DIR,
      number_of_steps=1, # For speed, we just do 1 epoch
      session_config=sess_config,        
      save_summaries_secs=1)
  
    print('Finished training. Final batch loss %d' % final_loss)

Will save model to /tmp/tensorboard/flowers
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= https://a62b9279.ngrok.io
inputs: Tensor("batch:0", shape=(32, 299, 299, 3), dtype=float32) targets: Tensor("batch:2", shape=(32,), dtype=int64)
TRAIN_DIR=/tmp/tensorboard, ls:None
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /tmp/tensorboard/flowers/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global step 1: loss = 1.5598 (2.252 sec/step)
INFO:tensorflow:Stopping Training.
INFO:tensorflow:Finished training! Saving model to disk.
Finished training. Final batch loss 1
